# Putting it all together

So far we have done the following on the prior Notebooks:

- **Notebook 01**: We loaded the Azure Search Engine with enriched PDFs in index: "cogsrch-index-files"
- **Notebook 02**: We loaded more information to the Search Engine this time using a CSV file with 52k rows/articles in index: "cogsrch-index-csv"
- **Notebook 03**: We added AzureOpenAI GPT models to enhance the the production of the answer bu using Utility Chains of LLMs
- **Notebook 04**: We added memory to our system in order to power a conversational Chat Bot
- **Notebook 05**: We introduced Agents and Tools in order to be able to solve a more complex task: ask questions to Tabular datasets
- **Notebook 06**: We used a Utility Chain in order to talk to a SQL Database directly

We are missing one more thing: **How do we glue all these features together into a very smart GPT Smart Search Engine Chat Bot?**

We want a virtual assistant for our company that can get the question, think what tool to use, then get the answer. The goal is that, regardless of the source of the information (Search Engine, Bing Search, SQL Database, CSV File, JSON File, etc), the Assistant can answer the question correctly using the right tool.

In this Notebook we are going to create that "brain" Agent, that will understand the question and use the right tool to get the answer from the right source.

Let's go..

In [1]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.utilities import BingSearchAPIWrapper
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import AgentType, Tool, initialize_agent, tool
from langchain.memory import ConversationBufferMemory
from langchain.tools import BaseTool

from IPython.display import Markdown, HTML, display  

def printmd(string):
    display(Markdown(string))

#custom libraries that we will use later in the app
from app.utils import DocSearchWrapper, CSVTabularWrapper, SQLDbWrapper

# Don't mess with this unless you really know what you are doing
AZURE_SEARCH_API_VERSION = '2021-04-30-Preview'
AZURE_OPENAI_API_VERSION = "2023-03-15-preview"
BING_SEARCH_URL = "https://api.bing.microsoft.com/v7.0/search"

MODEL_DEPLOYMENT_NAME = "gpt-35-turbo"

# Change these below with your own services credentials
AZURE_SEARCH_ENDPOINT = "Enter your Azure Cognitive Search Endpoint ..."
AZURE_SEARCH_KEY = "Enter your Azure Cognitive Search Key ..."
AZURE_OPENAI_ENDPOINT = "Enter your Azure OpenAI Endpoint ..."
AZURE_OPENAI_API_KEY = "Enter your Azure OpenAI Key ..."
BING_SUBSCRIPTION_KEY = "Enter your Azure Bing Search Subscription Key"

In [2]:
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["AZURE_SEARCH_KEY"] = AZURE_SEARCH_KEY
os.environ["AZURE_SEARCH_ENDPOINT"] = AZURE_SEARCH_ENDPOINT
os.environ["BING_SEARCH_URL"] = os.environ['BING_SEARCH_V7_ENDPOINT'] = BING_SEARCH_URL
os.environ["BING_SUBSCRIPTION_KEY"] = os.environ['BING_SEARCH_V7_SUBSCRIPTION_KEY'] = BING_SUBSCRIPTION_KEY
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["LANGCHAIN_HANDLER"] = "langchain"

### Get the Tools - Doc Search, CSV Agent, SQL Agent and  Web Search

In the file app/utils.py we create a wrapper Class for each of the Functionalities that we developed in prior Notebooks:

In [4]:
# DocSearchWrapper is our Custom Tool Class created for Azure Cognitive Search + OpenAI
indexes = ["cogsrch-index-files", "cogsrch-index-csv"]
doc_search = DocSearchWrapper(indexes=indexes,k=5, deployment_name=MODEL_DEPLOYMENT_NAME)

In [5]:
# BingSearchAPIWrapper is a langchain Tool class to use the Bing Search API
# We will use this to add capabilities to our Engine to search for current information
# https://www.microsoft.com/en-us/bing/apis/bing-web-search-api
www_search = BingSearchAPIWrapper(k=5)

In [6]:
## CSVTabularWrapper is a custom Tool class crated to Q&A over CSV files
file_url = "./data/all-states-history.csv"
csv_search = CSVTabularWrapper(path=file_url, deployment_name=MODEL_DEPLOYMENT_NAME)

In [7]:
## SQLDbWrapper is a custom Tool class created to Q&A over a MS SQL Database
server = '<YOUR_SERVER_NAME>.database.windows.net'
database = '<YOUR_DATABASE_NAME>'
username = '<YOUR_DB_USERNAME>'
password = '<YOUR_DB_PASSWORD>'

sql_search = SQLDbWrapper(server=server, database=database,
                          username=username, password=password, k=30,
                          deployment_name=MODEL_DEPLOYMENT_NAME)

### Test the Tools

In [8]:
# Test the Documents Search Tool with a question we know it doesn't have the knowledge for
printmd(doc_search.run("what is the weather today in Dallas?"))

No Results Found

In [9]:
# Test the Document Search Tool with a question that we know it has the answer for
printmd(doc_search.run("what are covid risk factors?"))

Custom token limit for gpt-35-turbo : 3000
Combined docs tokens count: 2500
Chain Type selected: stuff


The risk factors for COVID-19 severity and progression to critical illness include old age, male gender, high body mass index, and underlying comorbidities such as hypertension, diabetes, obesity, chronic lung diseases, heart, liver and kidney diseases, tumors, clinically apparent immunodeficiencies, local immunodeficiencies, such as early type I interferon secretion capacity, and pregnancy. Possible complications include acute kidney injury, coagulation disorders, and thromboembolism. Laboratory indicators of COVID-19 include lymphopenia, eosinopenia, elevated levels of white blood cell counts, liver enzymes, lactate dehydrogenase, creatine kinase, C-reactive protein, procalcitonin, proinflammatory cytokines such as interleukin (IL)-6, IL-1beta, Krebs von den Lungen-6 (KL-6), and ferritin. Risk factors associated with COVID-19 severity and death include comorbidities such as hypertension and diabetes, leukocytosis, neutrophilia, elevated levels of D-dimer, interleukin-6, and interleukin-10, and decreased levels of lymphocytes, CD3 cells, and CD4 cells. 
SOURCES: https://demodatasetsp.blob.core.windows.net/litcovid/train.csv

In [10]:
# Test the Bing Search Tool
printmd(www_search.run("What is the temperature today in Dallas?"))

<b>Dallas</b>, TX <b>Weather</b> Forecast | <b>AccuWeather</b> <b>Dallas</b>, TX <b>Weather</b> Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days. <b>Dallas</b>, TX <b>Weather</b> Forecast,... <b>Weather</b> <b>Today</b> <b>in Dallas</b>, TX 79° Feels Like 6:41 am 8:08 pm High / Low 81° / 56° Wind 10 mph Humidity 31% Dew Point 46° Pressure 29.89 in UV Index 9 of 10 Visibility 10 mi Moon Phase Waxing... Hourly <b>Weather</b> Forecast for <b>Dallas</b>, TX - <b>The Weather</b> Channel | <b>Weather</b>.com <b>Today</b> tenDay largeDeviceBannerText Don&#39;t Miss Severe <b>Weather</b> Takes Aim At Texas, Gulf Coast, Florida New COVID... <b>Dallas</b>, TX <b>Weather</b> Conditions | <b>Weather</b> Underground Sensor Network Maps &amp; Radar star Popular Cities San Francisco, CA warning56 °F Partly Cloudy Manhattan, NY warning52 °F Rain Schiller Park,...

In [11]:
# Test the SQL Search Tool
sql_search.run("How many people in total died california in july 2020?")



> Entering new SQLDatabaseChain chain...
How many people in total died california in july 2020?
SQLQuery:SELECT SUM([death]) as total_deaths FROM covidtracking WHERE state = 'CA' AND date LIKE '2020-07%'
SQLResult: [(229362.0,)]
Answer:There were 229362 people who died in California in July 2020.
Explanation: 
The query selects the sum of the 'death' column from the covidtracking table where the state is California and the date is in July 2020. The result is a single row with a single column, which gives the total number of deaths in California in July 2020.
> Finished chain.


"There were 229362 people who died in California in July 2020.\nExplanation: \nThe query selects the sum of the 'death' column from the covidtracking table where the state is California and the date is in July 2020. The result is a single row with a single column, which gives the total number of deaths in California in July 2020."

### Define our experts/tools from our main "brain" agent

In [12]:
tools = [
    Tool(
        name = "Current Search",
        func=www_search.run,
        description="useful for when you need to answer questions about current events."
    ),
    Tool(
        name = "Doc Search",
        func=doc_search.run,
        description="useful for when you need to answer questions about covid or computer science.",
        return_direct=True
    ),
    Tool(
        name = "SQL Tabular Search",
        func=sql_search.run,
        description="useful for when you need to answer questions about number of cases, deaths, hospitalizations, tests, people in ICU, people in Ventilator, in the United States related to Covid-19",
        return_direct=True
    ),
]

**Note**: Notice that since both the CSV file and the SQL Database have the same exact data, we are only going to use the SQLDBWrapper Tool

### Create a shared memory for the "brain" agent

In [13]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

### Initialize the brain agent

In [14]:
llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0.5, max_tokens=500)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
                               verbose=True, memory=memory)

### Let's talk to our GPT Smart Search Engine chat bot now

In [15]:
agent_chain.run(input="hi, how are you doing today Virtual Assistant?")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "I'm just a machine, so I don't have feelings, but I'm here to assist you. How can I help you today?"
}

> Finished chain.


"I'm just a machine, so I don't have feelings, but I'm here to assist you. How can I help you today?"

In [16]:
agent_chain.run(input="what is your name?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "My name is Virtual Assistant."
}
```

> Finished chain.


'My name is Virtual Assistant.'

In [17]:
agent_chain.run(input="what is the weather right now in 75240?")



> Entering new AgentExecutor chain...
{
    "action": "Current Search",
    "action_input": "weather 75240"
}
Observation: Current <b>Weather</b>. 11:41 AM. 61° F. RealFeel® 72°. RealFeel Shade™ 61°. Air Quality Fair. Wind ESE 4 mph. Wind Gusts 9 mph. Mostly cloudy More Details. Dallas <b>Weather</b> Forecasts. <b>Weather Underground</b> provides local &amp; long-range <b>weather</b> forecasts, weatherreports, maps &amp; tropical <b>weather</b> conditions for the Dallas area. Hourly <b>Weather</b> Forecast for Dallas, TX - The <b>Weather</b> Channel | <b>Weather</b>.com Today tenDay largeDeviceBannerText Don&#39;t Miss Severe <b>Weather</b> Takes Aim At Texas, Gulf Coast, Florida New COVID... 10 Day <b>Weather</b> - Richardson, TX As of 9:40 pm CDT Tonight --/ 51° 8% Thu 27 | Night 51° 8% E 3 mph Clear skies. Low 51F. Winds light and variable. Humidity 84% UV Index 0 of 10 Moonrise 12:36 pm...
Thought:{
    "action": "Final Answer",
    "action_input": "The current weather in 75240 i

'The current weather in 75240 is mostly cloudy with a temperature of 61°F and a RealFeel® temperature of 72°F. The air quality is fair, and the wind is coming from the east-southeast at 4 mph with gusts up to 9 mph.'

In [18]:
agent_chain.run(input="What is Microsoft latest news?")



> Entering new AgentExecutor chain...
```json
{
    "action": "Current Search",
    "action_input": "Microsoft latest news"
}
```
Observation: April 12, 2023 Announcing updates to <b>Microsoft</b> Cloud for Healthcare, as we add <b>new</b> value across the care continuum, including <b>new</b> features to help payors transform and <b>new</b> AI capabilities to help providers gain insights and improve outcomes. April 11, 2023 <b>Microsoft</b> announces quarterly earnings release date April 4, 2023 AI in Action April 4, 2023 <b>Microsoft</b> shares rose 9% in extended trading on Tuesday after the software maker issued fiscal third-quarter results and quarterly guidance that exceeded analysts’ predictions. Here’s how the... <b>MICROSOFT</b> CORP. 295.37. USD. +19.95 +7.24%. Alphabet Inc. ’s Google and <b>Microsoft</b> Corp., whose quarterly earnings each got a boost from their established search and cloud-computing businesses ... <b>Microsoft</b> It might not get the same kind of attenti

"Microsoft recently announced updates to their Cloud for Healthcare, including new features to help payors transform and new AI capabilities to help providers gain insights and improve outcomes. Additionally, Microsoft's quarterly earnings exceeded analysts' predictions, causing their shares to rise 9% in extended trading on Tuesday."

In [19]:
agent_chain.run(input="What medicine helps with Covid?")



> Entering new AgentExecutor chain...
{
    "action": "Doc Search",
    "action_input": "Medicines for Covid-19 treatment"
}Custom token limit for gpt-35-turbo : 3000
Combined docs tokens count: 1148
Chain Type selected: stuff

Observation: There are several candidate drugs being tested for COVID-19 treatment, including remdesivir, favipiravir, ribavirin, umifenovir, arbidol, lopinavir, ritonavir, baricitinib, hydroxychloroquine, nitazoxanide, azithromycin, baloxavir, oseltamivir, losartan, and tocilizumab. However, no drugs have been approved yet to treat infections with human coronaviruses. Clinical trials are ongoing to evaluate the efficacy of these drugs. It is important to administer off-label and investigational drugs in the setting of clinical trials to generate as much scientific knowledge as achievable while engaging in the best efforts to treat patients and save lives. 
SOURCES: https://demodatasetsp.blob.core.windows.net/litcovid/train.csv


> Finished chain.


'There are several candidate drugs being tested for COVID-19 treatment, including remdesivir, favipiravir, ribavirin, umifenovir, arbidol, lopinavir, ritonavir, baricitinib, hydroxychloroquine, nitazoxanide, azithromycin, baloxavir, oseltamivir, losartan, and tocilizumab. However, no drugs have been approved yet to treat infections with human coronaviruses. Clinical trials are ongoing to evaluate the efficacy of these drugs. It is important to administer off-label and investigational drugs in the setting of clinical trials to generate as much scientific knowledge as achievable while engaging in the best efforts to treat patients and save lives. \nSOURCES: https://demodatasetsp.blob.core.windows.net/litcovid/train.csv'

In [20]:
agent_chain.run(input="How many people died of covid in Texas in 2020?")



> Entering new AgentExecutor chain...
{
    "action": "SQL Tabular Search",
    "action_input": "Number of deaths in Texas due to covid-19 in 2020"
}

> Entering new SQLDatabaseChain chain...
Number of deaths in Texas due to covid-19 in 2020
SQLQuery:SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%' 
SQLResult: [(27437.0,), (27088.0,), (26762.0,), (26521.0,), (26472.0,), (26421.0,), (26408.0,), (26208.0,), (25900.0,), (25606.0,), (25415.0,), (25348.0,), (25226.0,), (24954.0,), (24660.0,), (24394.0,), (24142.0,), (23937.0,), (23911.0,), (23800.0,), (23551.0,), (23325.0,), (23081.0,), (22808.0,), (22627.0,), (22594.0,), (22502.0,), (22255.0,), (22000.0,), (21756.0,), (21549.0,), (21379.0,), (21357.0,), (21309.0,), (21207.0,), (21156.0,), (20950.0,), (20750.0,), (20588.0,), (20556.0,), (20467.0,), (20296.0,), (20113.0,), (19883.0,), (19696.0,), (19579.0,), (19559.0,), (19470.0,), (19320.0,), (19147.0,), (19004.0,), (18863.0,), (18769.0,), (18743.0,), (18700.0,),

"The number of deaths in Texas due to COVID-19 in 2020 was 27437.\nExplanation: \nI queried the covidtracking table for the 'death' column where the state is 'TX' and the date starts with '2020'. The query returned a list of tuples with the number of deaths for each day in 2020. I manually added up the numbers to get the total number of deaths in Texas due to COVID-19 in 2020, which is 27437."

In [21]:
agent_chain.memory.buffer

[HumanMessage(content='hi, how are you doing today Virtual Assistant?', additional_kwargs={}),
 AIMessage(content="I'm just a machine, so I don't have feelings, but I'm here to assist you. How can I help you today?", additional_kwargs={}),
 HumanMessage(content='what is your name?', additional_kwargs={}),
 AIMessage(content='My name is Virtual Assistant.', additional_kwargs={}),
 HumanMessage(content='what is the weather right now in 75240?', additional_kwargs={}),
 AIMessage(content='The current weather in 75240 is mostly cloudy with a temperature of 61°F and a RealFeel® temperature of 72°F. The air quality is fair, and the wind is coming from the east-southeast at 4 mph with gusts up to 9 mph.', additional_kwargs={}),
 HumanMessage(content='What is Microsoft latest news?', additional_kwargs={}),
 AIMessage(content="Microsoft recently announced updates to their Cloud for Healthcare, including new features to help payors transform and new AI capabilities to help providers gain insights

# Summary

Great!, We just built the GPT Smart Search Engine!
In this Notebook we created the brain, the decision making Agent that decides what Tool to use to answer the question from the user. This is what was necessary in order to have an smart chat bot.

We can have many tools to accomplish different tasks, including connecting to APIs, dealing with File Systems, and even using Humans as Tools. For more reference see [HERE](https://python.langchain.com/en/latest/modules/agents/tools.html)

# NEXT
Go back to your forked github repo and Deploy the web Application by following app/README.md